In [25]:
import gurobipy as gp
import pandas as pd
import ast

In [26]:
od_matrix = pd.read_csv('Limitation test/z6p12v3/Model_Input/od_matrix.csv', index_col=0)
canal_network = pd.read_csv('Limitation test/z6p12v3/Model_Input/canal_network.csv', index_col=0)
demand = pd.read_csv('Limitation test/z6p12v3/Model_Input/demand.csv')

# convert the column names to integers
od_matrix.columns = od_matrix.columns.astype(int)
canal_network.columns = canal_network.columns.astype(int)

od_matrix

,0,1,2,3,4,5
0,0,1,1,1,1,1
1,1,0,1,2,2,2
2,1,1,0,1,2,2
3,1,2,1,0,1,2
4,1,2,2,1,0,1
5,1,2,2,2,1,0


In [27]:
demand

,Zone_id,Period,Rental,Return
0,0,0,0,0
1,0,1,0,0
2,0,2,0,0
3,0,3,1,0
4,0,4,3,0
...,...,...,...,...
67,5,7,0,12
68,5,8,0,9
69,5,9,0,0
70,5,10,0,0


### Basic sets

In [29]:
available_vessel = 3
tot_periods = int(len(demand) / len(od_matrix)) 

# basic sets
V = list(range(available_vessel)) # set of vessels
Z = od_matrix.index.tolist() # set of zones
Z_S = canal_network.index.tolist() # set of zones with a candidate stop
T = list(range(tot_periods)) # set of time periods

N = [(z,t) for z in Z for t in T] # set of nodes
N_S = [(z,t) for z in Z_S for t in T] # set of nodes where vessels could be located
FIC = ['start', 'end'] # set of fictitious nodes


### Subsets

Arcs

In [30]:
##### for courier self rebalancing flows
# possible source nodes pointing to n
def gen_source_ns(n):
    possible_source_nodes = []
    for zone in Z:
        if zone == n[0] and n[1] != 0:
            possible_source_nodes.append((zone, n[1]-1))
        travel_time = od_matrix.iloc[zone, n[0]]
        if n[1]-travel_time >= 0:
            possible_source_nodes.append((zone, n[1]-travel_time))
    return possible_source_nodes

# possible sink nodes for n
def gen_sink_ns(n):
    reachable_nodes = []
    for zone in Z:
        if zone == n[0] and n[1] != tot_periods-1:
            reachable_nodes.append((zone, n[1]+1))
        travel_time = od_matrix.iloc[n[0], zone]
        if n[1]+travel_time <= tot_periods-1:
            reachable_nodes.append((zone, n[1]+travel_time))
    return reachable_nodes

# set of plus(n)
N_plus = {}
for n in N:
    N_plus[n] = gen_sink_ns(n)

# set of minus(n)
N_minus = {}
for n in N:
    N_minus[n] = gen_source_ns(n)

A = [(m, n) for n in N for m in N_minus[n]] # set of arcs
A1 = [(n,m) for n in N for m in N_plus[n]] # set of arcs

print([a for a in A if a not in A1])
print([a for a in A1 if a not in A])

[]
[]


Arcs for vessel movement

In [31]:
A_Fic = []
A_Fic_start = [('start', (0,t)) for t in T]
A_Fic_end = [((0,t), 'end') for t in T]
A_Fic = A_Fic_start + A_Fic_end


def sink_vessel(w):
    linked_nodes = []
    # get zones with a candidate stop that is linked to the zone of w
    linked_phy_stops = canal_network.columns[canal_network.loc[w[0],:] == 1].tolist()
    for stop in linked_phy_stops + [w[0]]:
        if w[1]+1 <= tot_periods-1:
            linked_nodes.append((stop, w[1]+1))
    if w[0] == 0:
        linked_nodes.append('end')
    return linked_nodes

def source_vessel(w):
    linked_nodes = []
    # get zones with a candidate stop that is linked to the zone of w
    linked_phy_stops = canal_network.columns[canal_network.loc[w[0],:] == 1].tolist()
    for stop in linked_phy_stops + [w[0]]:
        if w[1]-1 >= 0:
            linked_nodes.append((stop, w[1]-1))
    if w[0] == 0:
        linked_nodes.append('start')
    return linked_nodes

NS_plus = {}
for w in N_S:
    NS_plus[w] = sink_vessel(w)

NS_minus = {}
for w in N_S:
    NS_minus[w] = source_vessel(w)

# set of vessel movement arcs
A_S = []
for w in N_S:
    for node in sink_vessel(w):
        A_S.append((w, node))

# remove arcs that associated with the fictitious nodes
A_S = [a for a in A_S if a[0] != 'start' and a[1] != 'end']

A_S1 = []
for w in N_S:
    for node in source_vessel(w):
        A_S1.append((node, w))

A_S1 = [a for a in A_S1 if a[0] != 'start' and a[1] != 'end']
print([a for a in A_S if a not in A_S1])
print([a for a in A_S1 if a not in A_S])


[]
[]


## Model

### Variable

In [32]:
model = gp.Model('BikeRebalancing_arcBased')

# 1 if a vessel is used; otherwise, 0
delta = {}
for v in V:
    delta[v] = model.addVar(vtype=gp.GRB.BINARY, name=f'delta_{v}')

# 1 if a vessel arc a ∈ A_S A_FIC is used by a vessel v ∈ V, otherwise, 0
y = {}
for v in V:
    for a in A_S + A_Fic:
        y[a,v] = model.addVar(vtype=gp.GRB.BINARY, name=f'y_{a}_{v}')

# 1 if a vessel v ∈ V is located at node w ∈ N_S; otherwise, 0
phi = {}
for v in V:
    for w in N_S:
        phi[w,v] = model.addVar(vtype=gp.GRB.BINARY, name=f'phi_{w}_{v}')

# 1 if a stop z ∈ Z_S is open; otherwise, 0
zeta = {}
for z in Z_S:
    zeta[z] = model.addVar(vtype=gp.GRB.BINARY, name=f'zeta_{z}')

# non-negative integer, number of dropped bikes to node n ∈ N from vessel v ∈ V at node w ∈ N_minus(n) cap N_S
d = {}
for v in V:
    for n in N:
        for w in [x for x in N_minus[n] if x in N_S]:
            d[w,n,v] = model.addVar(vtype=gp.GRB.INTEGER, name=f'd_{w}_{n}_{v}')

# non-negative integer, number of collected bikes from node n ∈ N to vessel v ∈ V at node w ∈ N_plus(n) cap N_S
r = {}
for v in V:
    for n in N:
        for w in [x for x in N_plus[n] if x in N_S]:
            r[n,w,v] = model.addVar(vtype=gp.GRB.INTEGER, name=f'r_{n}_{w}_{v}')

# non-negative integer, number of bikes self-redistributed by couriers to node n ∈ N from node m ∈ N_plus(n)
g = {}
for a in A:
    g[a] = model.addVar(vtype=gp.GRB.INTEGER, name=f'g_{a}')

# non-negative integer, number of bikes borrowed to node n ∈ N from w ∈ N_minus(n) cap N_S
b = {}
for n in N:
    for w in [x for x in N_minus[n] if x in N_S]:
        b[w,n] = model.addVar(vtype=gp.GRB.INTEGER, name=f'b_{w}_{n}')

#  non-negative integer, number of bikes returned from node n ∈ N to the stop-time node w ∈ N_plus(n) cap N_S
q = {}
for n in N:
    for w in [x for x in N_plus[n] if x in N_S]:
        q[n,w] = model.addVar(vtype=gp.GRB.INTEGER, name=f'q_{n}_{w}')

# non-negative integer, the inventory level of the stop associated with stop-time node w ∈ N_S
p = {}
for w in N_S:
    p[w] = model.addVar(vtype=gp.GRB.INTEGER, name=f'p_{w}')

# non-negative integer, inventory level of vessel v ∈ V at stop-period node ∈ N_S
i = {}
for v in V:
    for w in N_S + FIC:
        i[w,v] = model.addVar(vtype=gp.GRB.INTEGER, name=f'i_{w}_{v}')



### Parameters

In [33]:
# ---------------------demand ---------------------
# demand in each zone at each period
Demands = {}
for n in N:
    zone_id = n[0]
    period = n[1]
    # return the value in the column of "Rental" by zone_id and period in df demand
    Demands[n] = int(demand.loc[(demand['Zone_id'] == zone_id) & (demand['Period'] == period), 'Rental'].values[0])

# return requests in each stop at each period
Returns = {}
for n in N:
    zone_id = n[0]
    period = n[1]
    Returns[n] = int(demand.loc[(demand['Zone_id'] == zone_id) & (demand['Period'] == period), 'Return'].values[0])

# ---------------------vessel and stop capacity ---------------------
# vessel capacity
vessel_capacity = 50

# stop capacity
stop_capacity = 5


# ---------------------cost ---------------------
vessel_cost = 500 # euro/vessel, single vessel cost
vessel_operation_cost = 0.15 # euro/unit operation time
stop_cost = 300  # euro/stop, cost of openning a stop
bike_cost = 250  # euro/bike, cost of purchasing a bike
courier_reblc_cost = 7.5  # euro/bike/unit distance, cost of asking couriers to serve one rental or return demand covering one unit distance

BigM = 200

### Objective

In [34]:
# quantity of rented vessels
vessel_quant = gp.quicksum(delta[v] for v in V)
vessel_operation_time = gp.quicksum(phi[w,v] for v in V for w in N_S)
open_stop_quant = gp.quicksum(zeta[z] for z in Z_S)
bike_buy = gp.quicksum(i[FIC[0],v] for v in V) + gp.quicksum(p[(z,0)] for z in Z_S)
bikeDist_handby_courier = gp.quicksum(g[m,n] * od_matrix.loc[m[0],n[0]] for n in N for m in N_minus[n])

bike_droppedby_vessel = gp.quicksum(d[w,n,v] for v in V for n in N for w in [x for x in N_minus[n] if x in N_S])
bike_pickupby_vessel = gp.quicksum(r[n,w,v] for v in V for n in N for w in [i for i in N_plus[n] if i in N_S])
return_satisfiedby_stop = gp.quicksum(q[n,w] for n in N for w in [i for i in N_plus[n] if i in N_S])
demand_satisfiedby_stop = gp.quicksum(b[w,n] for n in N for w in [x for x in N_minus[n] if x in N_S])

# flow cost 
flow_cost = 0.2 * (bike_droppedby_vessel + bike_pickupby_vessel) + 0.1 * (return_satisfiedby_stop + demand_satisfiedby_stop)

# objective function
obj = vessel_cost * vessel_quant + vessel_operation_cost * vessel_operation_time + stop_cost * open_stop_quant + bike_cost * bike_buy + courier_reblc_cost * bikeDist_handby_courier + flow_cost
# obj = vessel_cost * vessel_quant + vessel_operation_cost * vessel_operation_time + stop_cost * open_stop_quant + courier_reblc_cost * bikeDist_handby_courier


model.setObjective(obj, gp.GRB.MINIMIZE)
model.update()

### Constraints

In [35]:
# vessel could start at any time
con1 = {}
for v in V:
    con1[v] = model.addConstr(gp.quicksum(y[a,v] for a in A_Fic_start) - delta[v] == 0, name=f'con1_{v}')

# a vessel shift can start and end at any time
con2 = {}
for v in V:
    con2[v] = model.addConstr(gp.quicksum(y[a,v] for a in A_Fic_start) - gp.quicksum(y[a,v] for a in A_Fic_end) == 0, name=f'con2_{v}')

# vessel flow conservation
con3 = {}   
for v in V:
    for w in N_S:
        con3[w,v] = model.addConstr(gp.quicksum(y[(w,w_bar),v] for w_bar in NS_plus[w]) - gp.quicksum(y[(w_bar,w),v] for w_bar in NS_minus[w]) == 0, name=f'con3_{w}_{v}')

# node visit check
con4 = {}
for v in V:
    for w in N_S:
        con4[w,v] = model.addConstr(phi[w,v] <= gp.quicksum(y[(w,w_bar),v] for w_bar in NS_plus[w]) + gp.quicksum(y[(w_bar,w),v] for w_bar in NS_minus[w]), name=f'con4_{w}_{v}')

con4_1 = {}
for v in V:
    for w in N_S:
        con4_1[w,v] = model.addConstr(BigM * phi[w,v] >= gp.quicksum(y[(w,w_bar),v] for w_bar in NS_plus[w]) + gp.quicksum(y[(w_bar,w),v] for w_bar in NS_minus[w]), name=f'con4_{w}_{v}')

# stop selection 
con5_1 = {}
for z in Z_S:
    con5_1[z] = model.addConstr(gp.quicksum(phi[(z,t),v] for t in T for v in V) <= BigM * zeta[z], name=f'con5_1_{z}')

con5_2 = {}
for z in Z_S:
    con5_2[z] = model.addConstr(gp.quicksum(phi[(z,t),v] for t in T for v in V) >= zeta[z], name=f'con5_2_{z}')

# bikes dropoff flow activation
con6 = {}
for v in V:
    for n in N:
        for w in [x for x in N_minus[n] if x in N_S]:
            con6[w,n,v] = model.addConstr(d[w,n,v] - BigM * phi[w,v] <= 0, name=f'con6_{w}_{n}_{v}')


# bikes pickup flow activation
con7 = {}
for v in V:
    for n in N:
        for w in [x for x in N_plus[n] if x in N_S]:
            con7[n,w,v] = model.addConstr(r[n,w,v] - BigM * phi[w,v] <= 0, name=f'con7_{n}_{w}_{v}')

# demand satisfaction
con8 = {}
for n in N:
    con8[n] = model.addConstr(gp.quicksum(d[w,n,v] for v in V for w in [x for x in N_minus[n] if x in N_S]) + gp.quicksum(b[w,n] for w in [x for x in N_minus[n] if x in N_S]) + gp.quicksum(g[m,n] for m in N_minus[n]) == Demands[n], name=f'con8_{n}')


# return satisfaction
con9 = {}
for n in N:
    con9[n] = model.addConstr(gp.quicksum(r[n,w,v] for v in V for w in [i for i in N_plus[n] if i in N_S]) + gp.quicksum(q[n,w] for w in [i for i in N_plus[n] if i in N_S]) + gp.quicksum(g[n,m] for m in N_plus[n]) == Returns[n], name=f'con9_{n}')

con10_1 = {}
for v in V:
    for w in N_S:
        for w_bar in [x for x in NS_plus[w] if x != 'start']:
            con10_1[(w_bar,w),v] = model.addConstr(BigM * (1-y[(w,w_bar),v]) + i[w_bar,v] >= i[w,v] + gp.quicksum(r[n,w,v] for n in N_minus[w]) - gp.quicksum(d[w,n,v] for n in N_plus[w]), name=f'con10_1_{w_bar}_{w}_{v}')

con10_2 = {}
for v in V:
    for w in N_S:
        for w_bar in [x for x in NS_plus[w] if x != 'start']:
            con10_2[(w_bar,w),v] = model.addConstr(BigM * (y[(w,w_bar),v]-1) + i[w_bar,v] <= i[w,v] + gp.quicksum(r[n,w,v] for n in N_minus[w]) - gp.quicksum(d[w,n,v] for n in N_plus[w]), name=f'con10_2_{w_bar}_{w}_{v}')

con10_3 = {}
for v in V:
    for a in A_Fic_start:
        con10_3[a,v] = model.addConstr(BigM * (1-y[a,v]) + i[a[1],v] >= i[a[0],v], name=f'con10_3_{a}_{v}')

con10_4 = {}
for v in V:
    for a in A_Fic_start:
        con10_4[a,v] = model.addConstr(BigM * (y[a,v]-1) + i[a[1],v] <= i[a[0],v], name=f'con10_4_{a}_{v}')

# vessel inventory capacity
con11 = {}
for v in V:
    for w in N_S:
        con11[w,v] = model.addConstr(i[w,v] - phi[w,v] * vessel_capacity <= 0, name=f'con11_{w}_{v}')

con11_1 = {}
for v in V:
    for w in FIC:
        con11_1[w,v] = model.addConstr(i[w,v] - vessel_capacity <= 0, name=f'con11_1_{w}_{v}')

# stop inventory tracking
con12 = {}
for z in Z_S:
    for t in T[:-1]:
        con12[(z,t)] = model.addConstr(p[(z,t+1)] == p[(z,t)] + gp.quicksum(q[n,(z,t)] for n in N_minus[(z,t)]) - gp.quicksum(b[(z,t),n] for n in N_plus[(z,t)]), name=f'con12_{z}_{t}')
    for t in T[-1:]:
        con12[(z,t)] = model.addConstr(p[(z,t)] + gp.quicksum(q[n,(z,t)] for n in N_minus[(z,t)]) - gp.quicksum(b[(z,t),n] for n in N_plus[(z,t)]) - stop_capacity * zeta[z] <= 0, name=f'con12_{z}_{t}')

# stop service capability 
con13 = {}
for w in N_S:
    con13[w] = model.addConstr(gp.quicksum(q[n,w] for n in N_minus[w]) + gp.quicksum(b[w,n] for n in N_plus[w]) - BigM * zeta[w[0]] <= 0, name=f'con13_{w}')

# stop capacity
con14 = {}
for w in N_S:
    con14[w] = model.addConstr(p[w] - stop_capacity * zeta[w[0]] <= 0, name=f'con14_{w}')

# model.write("cosntrains.lp")

In [36]:
# set the time limit
model.setParam('TimeLimit', 7200)

# optimize the model
model.optimize()

# recorde the solving time
solving_time = model.Runtime

# model.computeIIS()
# model.write("model.ilp")

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5580 rows, 5421 columns and 43005 nonzeros
Model fingerprint: 0xaf4d0873
Variable types: 0 continuous, 5421 integer (1023 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e-01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 2466 rows and 3043 columns
Presolve time: 0.05s
Presolved: 3114 rows, 2378 columns, 19393 nonzeros
Variable types: 0 continuous, 2378 integer (1014 binary)

Root relaxation: objective 5.784048e+02, 2311 iterations, 0.05 seconds (0.08 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  578.40482    0  171          -  578.40

In [12]:
# print the objective value
print(f"Objective value = {model.objVal}")
print(f"Solving time = {solving_time}")


all_vars = model.getVars()
values = model.getAttr("X", all_vars)
names = model.getAttr("VarName", all_vars)

for name, val in zip(names, values):
    if val != 0:
        print(f"{name} = {val}")

Objective value = 19329.3
Solving time = 58.45910906791687
delta_0 = 1.0
delta_2 = 1.0
y_((0, 1), (0, 2))_0 = 1.0
y_((0, 2), (0, 3))_0 = 1.0
y_((0, 3), (0, 4))_0 = 1.0
y_((0, 4), (0, 5))_0 = 1.0
y_('start', (0, 1))_0 = 1.0
y_((0, 5), 'end')_0 = 1.0
y_((0, 0), (0, 1))_2 = 1.0
y_((0, 1), (0, 2))_2 = 1.0
y_((0, 2), (0, 3))_2 = 1.0
y_((0, 3), (0, 4))_2 = 1.0
y_('start', (0, 0))_2 = 1.0
y_((0, 4), 'end')_2 = 1.0
phi_(0, 1)_0 = 1.0
phi_(0, 2)_0 = 1.0
phi_(0, 3)_0 = 1.0
phi_(0, 4)_0 = 1.0
phi_(0, 5)_0 = 1.0
phi_(0, 0)_2 = 1.0
phi_(0, 1)_2 = 1.0
phi_(0, 2)_2 = 1.0
phi_(0, 3)_2 = 1.0
phi_(0, 4)_2 = 1.0
zeta_0 = 1.0
d_(0, 2)_(0, 2)_0 = 10.0
d_(0, 1)_(1, 2)_0 = 10.0
d_(0, 1)_(3, 2)_0 = 10.0
d_(0, 1)_(4, 2)_0 = 10.0
d_(0, 0)_(0, 1)_2 = 2.0
d_(0, 0)_(2, 1)_2 = 1.0
d_(0, 1)_(2, 2)_2 = 10.0
d_(0, 0)_(4, 1)_2 = 2.0
d_(0, 0)_(5, 1)_2 = 2.0
d_(0, 1)_(5, 2)_2 = 10.0
r_(0, 3)_(0, 4)_0 = 8.0
r_(1, 3)_(0, 4)_0 = 8.0
r_(1, 4)_(0, 5)_0 = 4.0
r_(2, 3)_(0, 4)_0 = 8.0
r_(3, 4)_(0, 5)_0 = 4.0
r_(4, 3)_(0, 4)_0 = 

In [101]:
vessel_quant_sol = sum(delta[v].X for v in V)
vessel_operation_time_sol = sum(y[a,v].X for v in V for a in A_S)
open_stop_quant_sol = sum(zeta[z].X for z in Z_S)
bike_buy_sol = sum(i[FIC[0],v].X for v in V) + sum(p[(z,0)].X for z in Z_S)
bikeDist_handby_courier_sol = sum(g[m,n].X for n in N for m in N_minus[n])

bike_droppedby_vessel = sum(d[w,n,v].X for v in V for n in N for w in [x for x in N_minus[n] if x in N_S])
bike_pickupby_vessel = sum(r[n,w,v].X for v in V for n in N for w in [i for i in N_plus[n] if i in N_S])
return_satisfiedby_stop = sum(q[n,w].X for n in N for w in [i for i in N_plus[n] if i in N_S])
demand_satisfiedby_stop = sum(b[w,n].X for n in N for w in [x for x in N_minus[n] if x in N_S])
self_rbl_flow = sum(g[a].X for a in A)

print(f"Total vessel used: {vessel_quant_sol}")
print(f"Total bike purchased: {bike_buy_sol}")
print(f"Total stop opened: {open_stop_quant_sol}")
print(f"Total bike dropped by vessel: {bike_droppedby_vessel}")
print(f"Total bike pickup by vessel: {bike_pickupby_vessel}")
print(f"Total demand satisfied by stop: {demand_satisfiedby_stop}")
print(f"Total return satisfied by stop: {return_satisfiedby_stop}")
print(f"Total bike self redistributed by couriers: {self_rbl_flow}")

print(f'Total demand: {sum(Demands.values())}')

Total vessel used: 2.0
Total bike purchased: 84.0
Total stop opened: 1.0
Total bike dropped by vessel: 79.0
Total bike pickup by vessel: 79.0
Total demand satisfied by stop: 5.0
Total return satisfied by stop: 5.0
Total bike self redistributed by couriers: 7.0
Total demand: 91
